# Recurrent neural network

Character-level language model

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg">

In [1]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

## Download data

In [2]:
isfile("input.txt") ||
  download("http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt", "input.txt")

true

## Preprocessing

In [3]:
text = collect(String(read("input.txt")))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

68-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [4]:
N = length(alphabet)
seqlen = 50
nbatch = 50

50

In [5]:
Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

1830-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 [[false false … false false; true false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; true false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false fals

## Model

In [6]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 128),
    Dense(128, N),
    softmax)

Chain(Recur(LSTMCell(68, 128)), Recur(LSTMCell(128, 128)), Dense(128, 68), NNlib.softmax)

## Loss function

In [7]:
function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

loss (generic function with 1 method)

## Optimizer

In [8]:
opt = ADAM(params(m), 0.01)
evalcb = () -> @show loss(Xs[5], Ys[5])

#5 (generic function with 1 method)

## Training

In [9]:
Flux.train!(loss, zip(Xs, Ys), opt, cb=throttle(evalcb, 30))

loss(Xs[5], Ys[5]) = 202.45528293464412 (tracked)
loss(Xs[5], Ys[5]) = 167.75381373898688 (tracked)
loss(Xs[5], Ys[5]) = 167.14824138044773 (tracked)
loss(Xs[5], Ys[5]) = 163.43542053861623 (tracked)
loss(Xs[5], Ys[5]) = 154.93595763773237 (tracked)
loss(Xs[5], Ys[5]) = 145.1697120910843 (tracked)
loss(Xs[5], Ys[5]) = 136.49327767515427 (tracked)
loss(Xs[5], Ys[5]) = 130.9153451688647 (tracked)
loss(Xs[5], Ys[5]) = 126.65639836622245 (tracked)
loss(Xs[5], Ys[5]) = 124.21062477185251 (tracked)
loss(Xs[5], Ys[5]) = 122.30418090567937 (tracked)
loss(Xs[5], Ys[5]) = 120.76118972360231 (tracked)
loss(Xs[5], Ys[5]) = 118.91092914991603 (tracked)
loss(Xs[5], Ys[5]) = 117.6279769838344 (tracked)
loss(Xs[5], Ys[5]) = 116.52415935235881 (tracked)
loss(Xs[5], Ys[5]) = 115.05858543676312 (tracked)
loss(Xs[5], Ys[5]) = 113.90204807897454 (tracked)
loss(Xs[5], Ys[5]) = 113.69725005157423 (tracked)
loss(Xs[5], Ys[5]) = 113.07024491715369 (tracked)
loss(Xs[5], Ys[5]) = 112.16889728181485 (tracked)
los

## Sampling

In [10]:
function sample(m, alphabet, len; temp=1)
    Flux.reset!(m)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, m(onehot(c, alphabet)).data)
    end
    return String(take!(buf))
end

sample (generic function with 1 method)

In [11]:
sample(m, alphabet, 1000) |> println

BONXBY:
Peondly, he hast ay me, he the valeptrac's from's
it be his man agrentlest abrand.

Servant:
So you could not digries Tortharly? He sumply,
And the effend them's yourself.

PERICLEA:
Profi's best they the eariSher friends,
More follow is me. Nor my grown expeeding virt.

QUEEN MARG EVALIN:
Shall have myself if touch empeny to
stay is san night. Verlontawed
O. There to salthind;
And he be ford, be York, reclailer; and to did tell sez;
The recent thy pizals he is to be hild that be:
But drut your your will be which the head,
My free to me here.

ROMEO:
When
Through thee he should marry chastes forse.

DUNREW:
Nay, you leave?

MACBETH:
What me.

MARIA:
He instincm, my chalf.
Then, in hand. I did eld this, I will Pescent
A winged, and told by thy where thine? Theressed and betther
Of this heart and ill his strange be do.

LUCILUS:
For some arm of leose his name:
A ore revenss.

Second Servil.N Coldame
Kild:
Ay, my inviten, what all your scever of
heart. Whire where; and m'stain so
